## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Подготовка данных

In [1]:
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
#from pyspark.ml import Pipeline




In [2]:
pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
        
RANDOM_SEED = 2022

spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True) 

In [3]:
df.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 2 rows



In [4]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
print(pd.DataFrame(df.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


In [6]:
df.describe().toPandas()

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20433,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,537.8705525375618,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,421.38507007403115,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


В исследуемых данных 20640 наблюдений, 10 показателей, 9 из них числовые, один категориальный признак.

In [7]:
columns = df.columns

for column in columns:
    check_col = F.col(column).cast('float').isNull()
    print(column, df.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 20640


In [8]:
median = df.approxQuantile('total_bedrooms', [0.5], 0)[0] 
df= df.na.fill({'total_bedrooms': median})

В колонке "total_bedrooms" выявлены 207 пропусков, заполнены медианным значением.

In [9]:
columns = df.columns

for column in columns:
    check_col = F.col(column).cast('float').isNull()
    print(column, df.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 20640


In [10]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

16418 4222


Разделение данных на тренировочную и тестовую часть в пропорции 4:1

In [11]:
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx'], handleInvalid="skip") 
indexer = indexer.fit(train_data)
train_data = indexer.transform(train_data)


encoder = OneHotEncoder(inputCols=['ocean_proximity_idx'],
                        outputCols=['ocean_proximity_ohe'])
encoder = encoder.fit(train_data)
train_data = encoder.transform(train_data)

train_data.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------

Обработка категориального признака - перевод в числовой.

In [12]:
categorical_assembler = \
        VectorAssembler(inputCols=["ocean_proximity_ohe"],
                                        outputCol="categorical_features")
train_data = categorical_assembler.transform(train_data)

Формирование вектора категориального признака.

In [13]:
train_data.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|
+----

In [14]:
numerical_cols  = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms',
                   'population', 'households', 'median_income']
target = 'median_house_value' 
train_data[numerical_cols].show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+
only showing top 2 rows



In [15]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
train_data = numerical_assembler.transform(train_data) 

In [16]:
train_data.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|[-124.35,40.54,52...|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0

In [17]:
scaler = StandardScaler(inputCol="numerical_features", outputCol="scaled_numerical_features",
                        withStd=True, withMean=False)
scalerModel = scaler.fit(train_data)
train_data = scalerModel.transform(train_data)
train_data.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|  numerical_features|scaled_numerical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+--------------------+-------------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                2.0|      (3,[2],[1.0])|       [0.0,0.0,1.0]|[-124.35,40.54,52...|     [-61.952887791441...|
|   -124.3|    4

Выделение числовых признаков, формирование общего вектора и масштабирование.

In [18]:
print(train_data.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'scaled_numerical_features']


In [19]:
all_features = ['categorical_features','scaled_numerical_features']

final_assembler_all = VectorAssembler(inputCols=all_features, 
                                  outputCol="all_features") 
train_data = final_assembler_all.transform(train_data)

train_data.select(all_features).show(3) 

+--------------------+-------------------------+
|categorical_features|scaled_numerical_features|
+--------------------+-------------------------+
|       [0.0,0.0,1.0]|     [-61.952887791441...|
|       [0.0,0.0,1.0]|     [-61.927977100733...|
|       [0.0,0.0,1.0]|     [-61.913030686308...|
+--------------------+-------------------------+
only showing top 3 rows



Формирование общего вектора числовых и категориальных признаков

In [20]:
print(train_data.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'scaled_numerical_features', 'all_features']


In [21]:
test_data = indexer.transform(test_data)
test_data = encoder.transform(test_data)
test_data = categorical_assembler.transform(test_data)
test_data = numerical_assembler.transform(test_data)
test_data = scalerModel.transform(test_data)
test_data = final_assembler_all.transform(test_data)

Предобработка тестовой выборки обученными на тренировочной части трансформерами.

In [22]:
test_data[all_features].show(2)

+--------------------+-------------------------+
|categorical_features|scaled_numerical_features|
+--------------------+-------------------------+
|       [0.0,0.0,1.0]|     [-61.927977100733...|
|       [0.0,0.0,1.0]|     [-61.893102133741...|
+--------------------+-------------------------+
only showing top 2 rows



# Промежуточный вывод 1

В изучаемом датасете 20640 наблюдений по 10 признакам (9 числовых и 1 категориальный). Данные корректны, с небольшим количеством пропусков (заменены медианным значением)

# Обучение моделей

In [23]:
lr_all = LinearRegression(labelCol=target, featuresCol='all_features')

model_all = lr_all.fit(train_data)


23/03/21 09:58:00 WARN Instrumentation: [afdffdc4] regParam is zero, which might cause numerical instability and overfitting.
23/03/21 09:58:00 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/21 09:58:00 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/03/21 09:58:01 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/03/21 09:58:01 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [24]:
lr_num = LinearRegression(labelCol=target, featuresCol='scaled_numerical_features')

model_num = lr_num.fit(train_data) 

23/03/21 09:58:03 WARN Instrumentation: [36894cef] regParam is zero, which might cause numerical instability and overfitting.


Обучены 2 модели линейной регрессии: одна обучена на числовых и категориальных признаках, другая только на числовых.

# Анализ результатов

In [25]:
pred_all = model_all.transform(test_data)
pred_all.select("all_features", "median_house_value", "prediction").show(5)


+--------------------+------------------+------------------+
|        all_features|median_house_value|        prediction|
+--------------------+------------------+------------------+
|[0.0,0.0,1.0,-61....|          103600.0|152990.34290591907|
|[0.0,0.0,1.0,-61....|           50800.0|214967.33394710347|
|[0.0,0.0,1.0,-61....|           58100.0|142714.13234324753|
|[0.0,0.0,1.0,-61....|           68400.0|132483.00813667197|
|[0.0,0.0,1.0,-61....|           72200.0| 164098.8109928714|
+--------------------+------------------+------------------+
only showing top 5 rows



In [26]:
pred_num = model_num.transform(test_data)
pred_num.select("scaled_numerical_features", "median_house_value", "prediction").show(5)

+-------------------------+------------------+------------------+
|scaled_numerical_features|median_house_value|        prediction|
+-------------------------+------------------+------------------+
|     [-61.927977100733...|          103600.0|101397.02412080672|
|     [-61.893102133741...|           50800.0|183325.89393649017|
|     [-61.883137857458...|           58100.0|109609.16753835836|
|     [-61.883137857458...|           68400.0| 80433.62265060423|
|     [-61.868191443033...|           72200.0| 129998.2662690985|
+-------------------------+------------------+------------------+
only showing top 5 rows



In [27]:
regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName="rmse")
rmse = regressionEvaluator.evaluate(pred_num)
print(f"The RMSE for the linear regression model is {rmse:0.2f}")

The RMSE for the linear regression model is 69009.76


In [28]:
mae = regressionEvaluator.setMetricName("mae").evaluate(pred_num)
print(f"The MAE for the linear regression model is {mae:0.2f}")

The MAE for the linear regression model is 50751.75


In [29]:
r2 = regressionEvaluator.setMetricName("r2").evaluate(pred_num)
print(f"The R2 for the linear regression model is {r2:0.2f}")

The R2 for the linear regression model is 0.65


In [30]:
regressionEvaluator = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName="rmse")
rmse = regressionEvaluator.evaluate(pred_all)
print(f'The RMSE for the linear regression model is {rmse:0.2f}')
#trainingSummary = model_all.summary
#print("RMSE: %f" % trainingSummary.rootMeanSquaredError)

The RMSE for the linear regression model is 68223.13


In [31]:
mae = regressionEvaluator.setMetricName("mae").evaluate(pred_all)
print(f"The MAE for the linear regression model is {mae:0.2f}")
#print("MAE: %f" % trainingSummary.meanAbsoluteError)

The MAE for the linear regression model is 49725.96


In [32]:
r2 = regressionEvaluator.setMetricName("r2").evaluate(pred_all)
print(f"The R2 for the linear regression model is {r2:0.2f}")
#print("R2: %f" % trainingSummary.r2)

The R2 for the linear regression model is 0.66


In [33]:
spark.stop()

# Вывод

На данных о жилой недвижимости в Калифорнии были были обучены две модели линейной регрессии для предсказания медианной стоимости дома в жилом массиве. Одна модель обучалась на всём массиве данных, другая обучалась без использования категориального признака: расположение относительно берега океана. Для оценки качества моделей были использованы метрики метрики RMSE, MAE и R2. Качество модели обученной на всех данных по всем метрикам показала лучшие результаты, однако разница оказалась незначительная. RMSE: 68223.13 и 69212.86, MAE: 49725.96 и 50866.56, R2: 0.66 и 0.65

# Примечание

Применение пайплайна для предобработки данных и применение RegressionEvaluator для оценки качества модели model_all (обученная на всём массиве данных) приводило к возникновению ошибки в функционировании SPARK, поэтому для оценки качества этой модели применён метод summary.